In [81]:
import torch
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [82]:
#exploration du dataset

df = pd.read_csv('../dataset.csv')

df.head(1)


,Unnamed: 0,url,title,description,cat1,cat2,cat3
0,0,https://www.tourisme-cambresis.fr/1-les-templiers-coffre.html,"Aventure-jeu : ""Les Templiers du coffre d'or""","Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque !\n\nConstruit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille.\n\nVous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",Jeu,Famille,Détente


In [83]:
#garder colonne description, cat1, cat2, cat3

df = df[['description', 'cat1', 'cat2', 'cat3']]

In [84]:
#supprimer les retours à la ligne dans la colonne description

df['description'] = df['description'].str.replace('\n', ' ')
df.head(1)


,description,cat1,cat2,cat3
0,"Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque ! Construit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille. Vous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",Jeu,Famille,Détente


In [85]:



#supprimer colonne cat2 et cat3

drop_columns = ['cat2', 'cat3']
df = df.drop(drop_columns, axis=1)

#supprimer le manquant dans la colonne description

df = df.dropna(subset=['description'])

#Labeencoder cat1

label_encoder = LabelEncoder()

df['cat1_encoded'] = label_encoder.fit_transform(df['cat1'])




array([14, 10,  8, 19, 13, 18,  9, 17,  1,  4,  3,  7,  2, 16, 12, 15,  0,
        6,  5, 11])

In [86]:
#initialise le tokenizer 

tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")


#tokenize les descriptions
inputs = tokenizer(df['description'].tolist(), padding=True, truncation=True, return_tensors="pt")

#convertir les labels en tensor

labels = torch.tensor(df['cat1_encoded'].tolist())

#Définir le nombre de classes

num_labels = len(df['cat1_encoded'].unique())

#charger le modèle
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#entrainer le modèle

dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

#définir l'optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

#entrainer le modèle
def train(model, dataloader, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, dataloader, optimizer)              

